# Setup
1. GitHub:
Create a github repository: gen-ai-medical-bot
Add .gitignore = Python
Select MIT license

2. Jupyter Lab:
Use the terminal and run command:
git clone https://github.com/lucho715/gen-ai-medical-chat-bot.git


